# PageRank

## Doba pretraživanja

google (and others)


* [50 milijardi stranica](http://www.worldwidewebsize.com/), [3.5 milijardi pretraga dnevno](http://www.internetlivestats.com/google-search-statistics/)
* __PageRank__
* povijest, kontekst - kolačići, spremanja podataka (o Vama), [200+ parametara](http://backlinko.com/google-ranking-factors)
[Mol11]: https://www.mathworks.com/moler/exm/chapters/pagerank.pdf "C, Moler, 'Google PageRank', mathWorks, 2011."

## PageRank

* Teorija grafova i linearna algebra
* [C. Moler, Google PageRank][Mol11]


[Mol11]: https://www.mathworks.com/moler/exm/chapters/pagerank.pdf "C, Moler, 'Google PageRank', mathWorks, 2011."

Neki programi:

* https://github.com/purzelrakete/Pagerank.jl

* https://gist.github.com/domluna/2b9358ccc89fee7d5e26

Probat ćemo primjer iz Molerovog članka.

In [1]:
i = vec([ 2 6 3 4 4 5 6 1 1])
j = vec([ 1 1 2 2 3 3 3 4 6])

9-element Array{Int64,1}:
 1
 1
 2
 2
 3
 3
 3
 4
 6

In [2]:
using SparseArrays
G=sparse(i,j,1.0)

6×6 SparseMatrixCSC{Float64,Int64} with 9 stored entries:
  [2, 1]  =  1.0
  [6, 1]  =  1.0
  [3, 2]  =  1.0
  [4, 2]  =  1.0
  [4, 3]  =  1.0
  [5, 3]  =  1.0
  [6, 3]  =  1.0
  [1, 4]  =  1.0
  [1, 6]  =  1.0

In [3]:
Matrix(G)

6×6 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0

In [4]:
c=sum(G,dims=1)
n=size(G,1)
for j=1:n
    if c[j]>0
        G[:,j]=G[:,j]/c[j]
    end
end

In [5]:
Matrix(G)

6×6 Array{Float64,2}:
 0.0  0.0  0.0       1.0  0.0  1.0
 0.5  0.0  0.0       0.0  0.0  0.0
 0.0  0.5  0.0       0.0  0.0  0.0
 0.0  0.5  0.333333  0.0  0.0  0.0
 0.0  0.0  0.333333  0.0  0.0  0.0
 0.5  0.0  0.333333  0.0  0.0  0.0

* $p$ je vjerojatnost da pratimo neki link
* $1-p$ je vjerojatnost da posjetimo slučajnu stranicu
* google koristi $p=0.85$ ?

In [6]:
p=0.85
δ=(1-p)/n

0.025000000000000005

In [7]:
z = ((1-p)*(c.!=0) + (c.==0))/n

1×6 Array{Float64,2}:
 0.025  0.025  0.025  0.025  0.166667  0.025

In [8]:
A=p*G+ones(n)*z

6×6 Array{Float64,2}:
 0.025  0.025  0.025     0.875  0.166667  0.875
 0.45   0.025  0.025     0.025  0.166667  0.025
 0.025  0.45   0.025     0.025  0.166667  0.025
 0.025  0.45   0.308333  0.025  0.166667  0.025
 0.025  0.025  0.308333  0.025  0.166667  0.025
 0.45   0.025  0.308333  0.025  0.166667  0.025

In [9]:
sum(A,dims=1)

1×6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0

## Ideja

Započnimo slučajnu šetnju iz vektora $x_0=\begin{bmatrix} 1/n \\ 1/n \\ \vdots \\ 1/n \end{bmatrix}$.

Sljedeći vektori su  

\begin{align*}
x_1&=A\cdot x_0 \\
x_2&=A\cdot x_1 \\
x_3&=A\cdot x_2\\
& \ \vdots
\end{align*}

Preslikavanje $A(x)=Ax$ nije kontrakcija u smislu Banachovog teorema o fiksnoj točci jer je $\|A\|_1=1$, ali se može pokazati da ima fiksnu točku. Također, ako je $x\geq 0$ (po komponentama), onda je $\|Ax\|_1=\|x\|_1$.

Kada se vektor _stabilizira_:

$$
A\cdot x\approx x,
$$

tada je $x[i]$ _rang stranice_ $i$.

In [10]:
using LinearAlgebra
function PageRank(G::SparseMatrixCSC{Float64,Int64},steps::Int)
    p=0.85
    c=sum(G,dims=1)/p
    n=size(G,1)
    for i=1:n
        G.nzval[G.colptr[i]:G.colptr[i+1]-1]./=c[i]
    end
    e=ones(n)
    x=e/n
    z = vec(((1-p)*(c.!=0) + (c.==0))/n)
    for j=1:steps
        x=G*x.+(z⋅x)
    end
    x
end

PageRank (generic function with 1 method)

In [11]:
fieldnames(typeof(G))

(:m, :n, :colptr, :rowval, :nzval)

In [12]:
G

6×6 SparseMatrixCSC{Float64,Int64} with 9 stored entries:
  [2, 1]  =  0.5
  [6, 1]  =  0.5
  [3, 2]  =  0.5
  [4, 2]  =  0.5
  [4, 3]  =  0.333333
  [5, 3]  =  0.333333
  [6, 3]  =  0.333333
  [1, 4]  =  1.0
  [1, 6]  =  1.0

In [13]:
G.colptr

7-element Array{Int64,1}:
  1
  3
  5
  8
  9
  9
 10

In [14]:
G.nzval

9-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5
 0.3333333333333333
 0.3333333333333333
 0.3333333333333333
 1.0
 1.0

In [15]:
G.rowval

9-element Array{Int64,1}:
 2
 6
 3
 4
 4
 5
 6
 1
 1

In [16]:
# Početni vektor
x=ones(n)/n

6-element Array{Float64,1}:
 0.16666666666666666
 0.16666666666666666
 0.16666666666666666
 0.16666666666666666
 0.16666666666666666
 0.16666666666666666

In [17]:
PageRank(G,15)

6-element Array{Float64,1}:
 0.3210244711000534
 0.17053803244044888
 0.10659550354273961
 0.13679458127849142
 0.06431030146206607
 0.2007371101762007

## [Stanford web graph](http://snap.stanford.edu/data/web-Stanford.html)

Nešto veći testni problem.

In [18]:
using DelimitedFiles
W=readdlm("../files/web-Stanford.txt",Int,comments=true)

2312497×2 Array{Int64,2}:
      1    6548
      1   15409
   6548   57031
  15409   13102
      2   17794
      2   25202
      2   53625
      2   54582
      2   64930
      2   73764
      2   84477
      2   98628
      2  100193
      ⋮  
 281849  165189
 281849  177014
 281849  226290
 281849  243180
 281849  244195
 281849  247252
 281849  281568
 281865  186750
 281865  225872
 281888  114388
 281888  192969
 281888  233184

In [19]:
?sparse;

In [20]:
S=sparse(W[:,2],W[:,1],1.0)

281903×281903 SparseMatrixCSC{Float64,Int64} with 2312497 stored entries:
  [6548  ,      1]  =  1.0
  [15409 ,      1]  =  1.0
  [17794 ,      2]  =  1.0
  [25202 ,      2]  =  1.0
  [53625 ,      2]  =  1.0
  [54582 ,      2]  =  1.0
  [64930 ,      2]  =  1.0
  [73764 ,      2]  =  1.0
  [84477 ,      2]  =  1.0
  [98628 ,      2]  =  1.0
  [100193,      2]  =  1.0
  [102355,      2]  =  1.0
  ⋮
  [119658, 281902]  =  1.0
  [166893, 281902]  =  1.0
  [168703, 281902]  =  1.0
  [180771, 281902]  =  1.0
  [266504, 281902]  =  1.0
  [275189, 281902]  =  1.0
  [44103 , 281903]  =  1.0
  [56088 , 281903]  =  1.0
  [90591 , 281903]  =  1.0
  [94440 , 281903]  =  1.0
  [216688, 281903]  =  1.0
  [256539, 281903]  =  1.0
  [260899, 281903]  =  1.0

In [21]:
@time x100=PageRank(S,100);

  1.223797 seconds (282.33 k allocations: 485.655 MiB, 21.15% gc time)


In [22]:
x101=PageRank(S,101);

In [23]:
maximum(abs,(x101-x100)./x101)

2.349137499752218e-7

In [24]:
# Ranks
sort(x100,rev=true)

281903-element Array{Float64,1}:
 0.011302854908405054
 0.009267830875148786
 0.008297272329247906
 0.0030231171615424336
 0.0030012789016376617
 0.002571731261436733
 0.0024537137277376732
 0.0024307906696690354
 0.002391045908292474
 0.0023640142856123114
 0.0023010006099533765
 0.002267415002261176
 0.002232447425626789
 ⋮
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7
 5.333685722320649e-7

In [25]:
# Pages
sortperm(x100,rev=true)

281903-element Array{Int64,1}:
  89073
 226411
 241454
 262860
 134832
 234704
 136821
  68889
 105607
  69358
  67756
 225872
 186750
      ⋮
 281627
 281646
 281647
 281700
 281715
 281728
 281778
 281785
 281813
 281849
 281865
 281888